## A notebook for testing headline processing
But this version used the OLD format of the NYT headlines, which were from news-api. Since then, NYT headliens were no longer served by NewsAPI so it needs a separate method. Then, sometime between January and April 2020, newsapi started serving from nytimes.com.

In [1]:
import json
import glob
import os
from datetime import datetime, timedelta
import pandas

In [2]:
def find_files(base_dir, begin_date, end_date):
    
    n_days = int((end_date - begin_date).total_seconds()/60/60/24)
    date_list = [begin_date + timedelta(days=x) for x in range(n_days+1)]
    
    search_strs = []
    
    for date in date_list:
        search_strs.append(date.strftime('%Y-%m-%d'))

    filenames=[]

    for search_str in search_strs:
        
        filenames += sorted(glob.glob(os.path.join(base_dir, '*' + search_str + '*.json')))
        
    return filenames

In [6]:
base_dir = "/home/will/Projects/headliner/datastore/raw/the-new-york-times-old/"
begin_date = datetime(2019, 12, 27)
end_date = datetime(2019, 12, 27)

In [7]:
filenames = find_files(base_dir, begin_date, end_date)

In [8]:
print(filenames[0])
with open(filenames[0], "r") as file:
    data = json.load(file)

/home/will/Projects/headliner/datastore/raw/the-new-york-times-old/the-new-york-times-2019-12-27T00:00:00-p001.json


In [9]:
filenames

['/home/will/Projects/headliner/datastore/raw/the-new-york-times-old/the-new-york-times-2019-12-27T00:00:00-p001.json',
 '/home/will/Projects/headliner/datastore/raw/the-new-york-times-old/the-new-york-times-2019-12-27T04:00:00-p001.json',
 '/home/will/Projects/headliner/datastore/raw/the-new-york-times-old/the-new-york-times-2019-12-27T08:00:00-p001.json',
 '/home/will/Projects/headliner/datastore/raw/the-new-york-times-old/the-new-york-times-2019-12-27T08:00:00-p002.json',
 '/home/will/Projects/headliner/datastore/raw/the-new-york-times-old/the-new-york-times-2019-12-27T12:00:00-p001.json',
 '/home/will/Projects/headliner/datastore/raw/the-new-york-times-old/the-new-york-times-2019-12-27T16:00:00-p001.json',
 '/home/will/Projects/headliner/datastore/raw/the-new-york-times-old/the-new-york-times-2019-12-27T20:00:00-p001.json']

In [100]:
type(data)

dict

In [114]:
from collections import Mapping
import json

with open(filenames[0], "r") as file:
    original = json.load(file)

with open(filenames[1], "r") as file:
    addition = json.load(file)

def merge_json(original, addition):

    for key, value in addition.items():
        if key in original:
            original_value = original[key]
            if isinstance(value, Mapping) and isinstance(original_value, Mapping):
                merge_dicts(original_value, value)
            elif not (isinstance(value, Mapping) or 
                      isinstance(original_value, Mapping)):
                original[key] = value
            else:
                raise ValueError('Attempting to merge {} with value {}'.format(
                    key, original_value))
        else:
            original[key] = value
    
    return original

In [115]:
original

{'status': 'ok',
 'totalResults': 8,
 'articles': [{'source': {'id': 'the-new-york-times',
    'name': 'The New York Times'},
   'author': 'Deb Amlen',
   'title': 'Take for a Spin',
   'description': '— Now this is the way to slide into a weekend, particularly the last one of the year. Sam Trabucco debuts a whopping 10 entries and even the stuff we’ve seen before is delightful. It’s not the most difficult Friday puzzle I’ve ever solved, but that’s meant to…',
   'url': 'https://www.nytimes.com/2019/12/26/crosswords/daily-puzzle-2019-12-27.html',
   'urlToImage': 'https://static01.nyt.com/images/2019/12/27/crosswords/27wordplay-racecar/merlin_24220895_b7bfa44b-4bd5-4dae-bde4-b4fb294e1bab-facebookJumbo.jpg',
   'publishedAt': '2019-12-27T03:56:00Z',
   'content': '8D. Nicely done. A Father figure? is a DAD BOD.\r\n46D. The clue Wedding circle? sounds as if it might be hinting at a wedding ring, but not in this puzzle. This is a Jewish wedding, and we are dancing the HORA, which is perfo

In [113]:
addition

{'status': 'ok',
 'totalResults': 7,
 'articles': [{'source': {'id': 'the-new-york-times',
    'name': 'The New York Times'},
   'author': 'Tyler Kepner',
   'title': 'Larry Walker Played Like a Hall of Famer, Through Thick and Thin Air',
   'description': 'Never mind the benefits of playing in Denver: Walker’s all-around performance is worth a pass to the Hall in his final year of eligibility.',
   'url': 'https://www.nytimes.com/2019/12/27/sports/baseball/larry-walker-hall-of-fame.html',
   'urlToImage': 'https://static01.nyt.com/images/2019/12/27/sports/26kepner-hof1-print/merlin_166357926_ebf6abc7-1b1c-499d-8fbc-2e5a8632c673-facebookJumbo.jpg',
   'publishedAt': '2019-12-27T07:00:08Z',
   'content': 'Switching from thin air to sea level all season takes a toll on the body, Weiss said, and the Rockies are the only team that must leave its time zone for every road game. Hitters can struggle on the road because pitches behave differently, he added; the same … [+1941 chars]'},
  {'sour

In [111]:
merge_json(original, addition)

{'status': 'ok',
 'totalResults': 7,
 'articles': [{'source': {'id': 'the-new-york-times',
    'name': 'The New York Times'},
   'author': 'Tyler Kepner',
   'title': 'Larry Walker Played Like a Hall of Famer, Through Thick and Thin Air',
   'description': 'Never mind the benefits of playing in Denver: Walker’s all-around performance is worth a pass to the Hall in his final year of eligibility.',
   'url': 'https://www.nytimes.com/2019/12/27/sports/baseball/larry-walker-hall-of-fame.html',
   'urlToImage': 'https://static01.nyt.com/images/2019/12/27/sports/26kepner-hof1-print/merlin_166357926_ebf6abc7-1b1c-499d-8fbc-2e5a8632c673-facebookJumbo.jpg',
   'publishedAt': '2019-12-27T07:00:08Z',
   'content': 'Switching from thin air to sea level all season takes a toll on the body, Weiss said, and the Rockies are the only team that must leave its time zone for every road game. Hitters can struggle on the road because pitches behave differently, he added; the same … [+1941 chars]'},
  {'sour

In [109]:
type(addition)

dict